In [ ]:
import numpy as np
import pandas as pd
import multiprocessing

import matplotlib.pyplot as plt
import bokeh
import bokeh.io
from bokeh.plotting import figure
from bokeh.io import output_notebook, show

import seaborn as sns

import re
import math
import copy

from collections import defaultdict
import csv
import itertools
import datetime 
from datetime import datetime
import time
import dateutil.parser
import pickle
import random

import gc
import zipfile
import sys, getopt
import os

from IPython.core.interactiveshell import InteractiveShell
from io import StringIO

import dask.dataframe as dd

InteractiveShell.ast_node_interactivity = "all"

%matplotlib inline
%config InlineBackend.figure_formats = {'png', 'retina'}

# Set up Bokeh for inline viewing
bokeh.io.output_notebook()

import dask.dataframe as ddf
import dask.array as da

pd.set_option('max_columns', 500)
pd.set_option('max_rows', 800)

import scipy


In [ ]:
def initialize_parameters(par = np.array([0.5, 0.9, 0, 1, 0])):
    # np.random.seed(3)
    parameters = {}

    parameters['alpha'] = par[0]
    parameters['beta'] = par[1]
    parameters['omega'] = par[2]* (1-par[1])    # one way to choose that is omega/(1-beta) = unconditional mean 
    parameters['sigma'] = par[3]
    parameters['f0'] = par[4]                   # one way to choose is unconditional mean 

    return parameters

def loglik(y, f, x, sigma):
    ll = -1/2*np.log(2*np.pi ) - 1/2*np.log(sigma) - 1/(2*sigma)*(y - x*f)**2 
    return ll


def score_compute(y, f, x, parameters, epsilon = 1e-7 ):
    alpha = parameters["alpha"]
    beta = parameters["beta"]
    omega = parameters['omega']
    sigma = parameters["sigma"]
    f0 = parameters["f0"]
    
    score = (y - x*f)/sigma
    # score = (y - x*f)
    
    return score

def filterGAS(y, x, parameters):
    
    alpha = parameters["alpha"]
    beta = parameters["beta"]
    omega = parameters['omega']
    sigma = parameters["sigma"]
    f0 = parameters["f0"]
    score0 = score_compute(y[0,:],  f0, x[0,:], parameters, epsilon = 1e-7) 
    f = np.zeros((len(y),1))
    
    f[0,:] = f0
    for t in range(1,len(y)):
        scoret = score_compute(y[t-1,:], f[t-1,:], x[t-1,:], parameters, epsilon = 1e-7) 
        f[t,:] = omega + alpha*scoret + beta*f[t-1,:] 

    return f

def loglikest(par, y, x):
    parameters = initialize_parameters(par)
    alpha = parameters["alpha"]
    beta = parameters["beta"]
    sigma = parameters["sigma"]
    # f0 = parameters["f0"]
    
    f = filterGAS(y, x, parameters) 
    ll = np.zeros((len(y), 1))
    m = len(y)

    for t in range(0, len(y)):
         ll[t,:] = loglik(y[t,:], f[t,:], x[t,:], sigma)
    loglik_res = -(np.sum(ll))/m
        
#     else:
#         loglik_res=10**9 # causing gradient problems??

    return loglik_res

In [ ]:
def score_compute_2(y, f, x, parameters, epsilon = 1e-7 ):
    alpha = parameters["alpha"]
    beta = parameters["beta"]
    omega = parameters['omega']
    sigma = parameters["sigma"]
    f0 = parameters["f0"]
    
    score = (y - x*f) # ** The 'type = 2' modification **
    
    return score

def filterGAS_2(y, x, parameters):
    
    alpha = parameters["alpha"]
    beta = parameters["beta"]
    omega = parameters['omega']
    sigma = parameters["sigma"]
    f0 = parameters["f0"]
    score0 = score_compute_2(y[0,:],  f0, x[0,:], parameters, epsilon = 1e-7) 
    f = np.zeros((len(y),1))
    
    f[0,:] = f0
    for t in range(1,len(y)):
        scoret = score_compute_2(y[t-1,:], f[t-1,:], x[t-1,:], parameters, epsilon = 1e-7) 
        f[t,:] = omega + alpha*scoret + beta*f[t-1,:] 

    return f

def loglikest_2(par, y, x):
    parameters = initialize_parameters(par)
    alpha = parameters["alpha"]
    beta = parameters["beta"]
    sigma = parameters["sigma"]
    # f0 = parameters["f0"]
    
    f = filterGAS_2(y, x, parameters) 
    ll = np.zeros((len(y), 1))
    m = len(y)

    for t in range(0, len(y)):
         ll[t,:] = loglik(y[t,:], f[t,:], x[t,:], sigma)
    loglik_res = -(np.sum(ll))/m
        
#     else:
#         loglik_res=10**9 # causing gradient problems??

    return loglik_res

In [267]:
def GAS_est(df):
    
    y = df.net_qty.values          # observed demand (response)
    x = df.buy_availability.values # buy_availability (explanatory)

    y = y.reshape((len(y),1)) 
    x = x.reshape((len(y),1))
    
    
    
    ret = pd.DataFrame()
    ret['year'] = df['year']
    ret['week'] = df['week']
    
    
    
    abc = scipy.optimize.minimize(
        loglikest_2,                                       # function to minimize (log likelihood y|x,theta)
        np.array([0.8, 0.9, np.mean(y), 1, np.mean(y)]),   # initial parameter values (starting)
        args=(y, x), 
        options ={'eps':1e-09, 'maxiter': 600, 'ftol': 1e-12},
        method='L-BFGS-B', 
        bounds=((0,  None),             # alpha
                (-1, 1),                # beta
                (0.001, np.mean(y)*2),  # omega 
                (0.001, None),          # sigma
                (0.001, np.mean(y)*2)   # f
               )
            )
        
    
    # --- CONVERGENCE control flow ---
    if abc.success == True:
        x1par = initialize_parameters(abc.x) 
        f_est = filterGAS_2(y, x, x1par)
        ret['f_est'] = f_est # pd.Series(f_est.reshape(len(y))).round(2)
        ret['Convergence'] = [abc.success] * len(y)
        ret['Convg type'] = ['Two'] * len(y)
        
    # **Modification if first algorithm fails
    elif abc.success == False:
        abc = scipy.optimize.minimize(
            loglikest,                                       # function to minimize (log likelihood y|x,theta)
            np.array([0.8, 0.9, np.mean(y), 1, np.mean(y)]), # initial parameter values (starting)
            args=(y, x), 
            options ={'eps':1e-09, 'maxiter': 600, 'ftol': 1e-12},
            method='L-BFGS-B', 
            bounds=((0,  None),             # alpha
                    (-1, 1),                # beta
                    (0.001, np.mean(y)*2),  # omega 
                    (0.001, None),          # sigma
                    (0.001, np.mean(y)*2)   # f
                   )
        )
        x1par = initialize_parameters(abc.x) 
        f_est = filterGAS(y, x, x1par)
        ret['f_est'] = f_est # pd.Series(f_est.reshape(len(y))).round(2)
        ret['Convergence'] = [abc.success] * len(y)
        ret['Convg type'] = ['One'] * len(y)

    return ret

# Apply GAS models

In [ ]:
dat0 = pd.read_csv('ch4k_df_eu.csv', low_memory = False, index_col = 0)

In [268]:
dat = dat0.copy()
dat.reset_index(inplace = True)
dat = dat[(dat.season == 'SS19') & (dat.country == 'EU') & (dat.season_net_qty > 1000)]

dat.sort_values(['article_number', 'year', 'week'], inplace = True)

dat.set_index(['article_number'], inplace = True)


In [269]:
# dat.shape 39525 rows
dat.dropna(inplace=True)
# dat.shape 37516 rows

len(dat.index.unique()) # articles: from 1627 to 1583

1583

In [ ]:
%%time

# size = 100: 35 s
# size = 500: 1min 21s
# size = 1000: 2min 31s
# size = 1627: 4min 13s
# size = 1583, 600 max iterations -- 8min 25s

#    --- SUBSET ---
# a = np.random.choice(dat.index.unique(), size = , replace = False)
# dat_samp = dat.loc[a ,:].copy()
# dat_samp.reset_index(inplace = True)
# ests = dat_samp.groupby('article_number').apply(GAS_est)
#    ---- ---- ----

f_ests = dat.groupby('article_number').apply(GAS_est)

In [ ]:
f_ests.index.shape
t = f_ests.index.droplevel()
f_ests = f_ests.set_index(t).reset_index()

f_ests.head()
f_ests.groupby(['Convergence', 'Convg type'])['article_number'].describe()

In [ ]:
dat2 = pd.merge(dat.reset_index(), f_ests, 
                left_on = ['article_number', 'year', 'week'], 
                right_on = ['article_number', 'year', 'week'])

dat2.shape
dat2.head()

# NCs = dat2[dat2.Convergence == False]
# NCs.shape
# len(NCs.article_number.unique())
# NCs.head()

dat2[dat2['Convg type'] == 'Two'].article_number.unique()

In [ ]:
# eg = np.random.choice(dat2.article_number.unique(), size = 1)
# eg[0]

eg = np.random.choice(dat2[dat2['Convg type'] == 'Two'].article_number.unique(), size = 1)
eg[0]

eg_plot = dat2[dat2.article_number == eg[0]]

eg_plot.shape
print('Estimated actual:', round(np.sum(eg_plot.f_est))) # estimated total actual demand
print('Observed:', np.sum(eg_plot.net_qty)) # total observed demand

# eg_plot.head()

# --- plots ---
pd.DataFrame(data = eg_plot[['net_qty', 'f_est']]).plot(linewidth = 4) # plot against week instead
plt.title('Observed & Estimated Gross Demand Quantity')

pd.DataFrame(eg_plot.buy_availability).plot(linewidth = 4)
plt.ylim(0, 1.05)
plt.title('Buy Availability')
plt.ylabel('Real Data')


## non-convergence EDA

In [ ]:
dat_nc = dat2[dat2.article_number == 'B44882']

y = dat_nc.net_qty.values # observed demand -- basically the response variable
x = dat_nc.buy_availability.values # basically the explanatory variable

y = y.reshape((len(y),1)) 
x = x.reshape((len(y),1))

abc = scipy.optimize.minimize(
    loglikest, # function to minimize (log likelihood y|x,theta)
    np.array([0.8, 0.9, np.mean(y), 1, np.mean(y)]), # initial parameter values (starting)
    args=(y, x), 
    options ={'eps':1e-09, 'maxiter': 2000, 'ftol': 1e-12},
    method='L-BFGS-B', 
    bounds=((0,  None),             # alpha
            (-1, 1),                # beta
            (0.001, np.mean(y)*2),  # omega 
            (0.001, None),          # sigma
            (0.001, np.mean(y)*2)   # f
           )
)

abc

In [ ]:
# ------------------------- Appendix --------------------------- 

In [ ]:
# def test(a, b):
#     ret = pd.DataFrame()
#     ret["A"] = [1, 2, 3]
#     ret['B'] = [b] * 3
#     return ret

# test(5, 9)

In [ ]:
# a = np.random.choice(dat.index.unique(), size = 1, replace = False)
# print('Article:', a[0])

# dat_a = dat[(dat.index == a[0])].copy()

# dat_a.sort_values(by = ['year', 'week'], inplace = True)

# print()
# print('Number of weeks:', dat_a['year'].count())
# print()

# y = dat_a.net_qty.values # observed demand -- basically the response variable
# x = dat_a.buy_availability.values # basically the explanatory variable

# y = y.reshape((len(y),1)) 
# x = x.reshape((len(y),1))


# x0 = np.array([0.8, 0.9, np.mean(y), 1, np.mean(y)]) # starting value for optimisation
#             # alpha, beta, omega, sigma, f0
#             # See initialize_parameters for ordering
        
# # # function to minimize log likelihood of observed values given parameters
# res = scipy.optimize.minimize(loglikest, 
#                               x0,        # initial parameter values (starting)
#                               args=(y, x), 
#                                   # y1: obs demand
#                                   # x: buy_availability
#                               options ={'eps':1e-09, 'maxiter': 200, 'ftol': 1e-12},
#                               method = 'L-BFGS-B', # 'TNC', , 'SLSQP'
#                               bounds =(
#                                   (0,  None),             # alpha
#                                   (-1, 1),                # beta
#                                   (0.001, np.mean(y)*2),  # omega 
#                                   (0.001, None),          # sigma
#                                   (0.001, np.mean(y)*2)   # f0
#                               ),
#                              )
# x1 = res.x.round(1) # numpy.ndarray, (5,)

# print('Message:', res.message)
# print()
# print('Iterations:', res.nit)
# print()
# print('Param est conv\'g:', res.success)
# print()
# print('alpha:', x1[0])
# print('beta:', x1[1])
# print('omega:', x1[2])
# print('sigma:', x1[3])
# print('f0:', x1[4])
# print()

In [ ]:
# # Get estimated parameters and filter out the demand

# x1par = initialize_parameters(res.x)
# # x1par = initialize_parameters(x0)
# f_est = filterGAS(y, x, x1par)

# d = pd.DataFrame({'Obs_demand': y[:,0], 'Est_true_demand': f_est[:,0]})

# # total real demand
# print('Estimated true demand:', round(np.sum(f_est)))

# # total observed demand
# print('Observed demand:', np.sum(y))

# # plt.rcParams["figure.figsize"] = [10,5]

# pd.DataFrame(data = d).plot(linewidth = 5)
# # plt.ylim(0, 1200)
# plt.title('Net Demand Quantity: Observed & Estimated', size = 22)

# pd.DataFrame(x).plot(linewidth = 5)
# plt.title('Buy Availability', size = 22)
# # plt.ylabel('Real Data')

# plt.xlabel('Week', size = 22)

In [ ]:
# dat.reset_index(inplace=True)

# a = pd.DataFrame(dat.groupby('article_number')['net_qty'].sum())
# b = dat[['article_number', 'season_net_qty']].drop_duplicates()
# c = pd.merge(a, b, left_index=True, right_on='article_number')
# c[np.abs(c.net_qty - c.season_net_qty) > 100]

In [265]:


1804 + 4616
58769 + 225574

58769 - 1804
220958 - 4616

220958 + 58769

(1804/58769)*0.07/(6420/(279727+6420))

6420

284343

56965

216342

279727